In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC mutation dataset (DNA_REPi)

os.chdir(dataset_dir + "/mutations/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations


In [9]:
# read GDSC mutation dataset (DNA_REPi)

GDSCM = pd.read_csv("GDSC_mutations.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCM.drop_duplicates(keep='last')
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:, ~GDSCM.columns.duplicated()]

GDSCM.head()

,143872,728577,2,2050,10243,79365,256006,4609,118788,113675,...,26517,28972,53917,283507,514,339210,64981,7335,5739,83442
683665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# change directory to read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [12]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [13]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Doxorubicin)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Doxorubicin)

TCGAEdoxorubicin = pd.read_csv("TCGA_exprs.Doxorubicin.eb_with.GDSC_exprs.Doxorubicin.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEdoxorubicin = pd.DataFrame.transpose(TCGAEdoxorubicin)
TCGAEdoxorubicin.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-3B-A9I0-01A,4.28791509210347,5.7087700970711,5.54045826719573,3.05637075605351,2.24363705677548,3.25982251227013,8.49998617466154,2.73915290143961,8.78300900393896,3.34683211634676,...,9.58808719294554,3.0241254728314,2.97779500953892,3.68506348928701,2.89347815187099,2.92295237753598,2.92448373105139,2.93379072110733,5.22289120271932,3.21724915174761
TCGA-3B-A9I1-01A,4.0585422649983,3.31781963303799,8.60059130074378,4.02605522787784,6.47977729163315,3.25982251227013,8.35296462583893,2.70390620942745,10.2892935732714,3.52657584490066,...,9.17906591914891,3.0241254728314,2.97779500953892,3.30867142457444,2.89347815187099,2.87783353779556,2.79579181836739,3.59727260527723,4.65344019134805,4.64481625794141
TCGA-CE-A481-01A,3.93912509179632,3.01355610282141,6.65333239972603,2.93917728336359,3.32166319866546,3.7386580396915,8.61970764126633,2.78132950016492,8.57238405965353,5.62387842220403,...,10.0426845542463,3.0241254728314,2.97779500953892,3.57588363901358,3.02526306371745,2.61668386187983,2.6312133532773,2.93379072110733,4.54946987256871,4.40460504435084


In [15]:
# change directory to read TCGA mutation dataset (Doxorubicin)

os.chdir(dataset_dir + "mutations")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations'

In [16]:
# read TCGA mutation dataset (Doxorubicin)

TCGAMdoxorubicin = pd.read_csv("TCGA_mutations.Doxorubicin.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGAMdoxorubicin.drop_duplicates(keep='last')
TCGAMdoxorubicin = pd.DataFrame.transpose(TCGAMdoxorubicin)
TCGAMdoxorubicin = TCGAMdoxorubicin.loc[:, ~TCGAMdoxorubicin.columns.duplicated()]

TCGAMdoxorubicin.head(3)

,285313,3845,317701,23177,197257,55690,2060,2956,9739,60682,...,114335,55897,2787,55049,94107,92815,400569,387103,728226,728689
TCGA-3B-A9I0-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.05,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-3B-A9I1-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-CE-A481-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# variance threshold for GDSC expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC mutation dataset (DNA_REPi)

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1

In [19]:
# select shared genes between GDSC and TCGA expression and mutation datasets 

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(TCGAEdoxorubicin.columns)
ls = ls.intersection(TCGAMdoxorubicin.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA # of common genes: {ls.shape[0]}")

GDSC and TCGA # of common genes: 17149


In [20]:
# select shared samples between GDSC expression, mutation, and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [21]:
# subset shared genes and samples in GDSC expression, mutation, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCM = GDSCM.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCM.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (801, 17149)
GDSC # of common samples and genes (Mutation): (801, 17149)
GDSC # of samples (Response): 9435


In [22]:
# select shared samples between TCGA expression and mutation datasets (Doxorubicin) 

ls3 = TCGAEdoxorubicin.index.intersection(TCGAMdoxorubicin.index)

print(f"TCGA # of common samples (Doxorubicin): {ls3.shape[0]}")

TCGA # of common samples (Doxorubicin): 18


In [23]:
# select shared genes and samples between TCGA expression and mutation datasets (Doxorubicin)

TCGAEdoxorubicin = TCGAEdoxorubicin.loc[ls3, ls]
TCGAMdoxorubicin = TCGAMdoxorubicin.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Doxorubicin): {TCGAEdoxorubicin.shape}")
print(f"TCGA # of common samples and genes for Mutation (Doxorubicin): {TCGAMdoxorubicin.shape}\n")

TCGA # of common samples and genes for Expression (Doxorubicin): (18, 17149)
TCGA # of common samples and genes for Mutation (Doxorubicin): (18, 17149)



In [24]:
# change directory to read DEGs (DNA_REPi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [25]:
# read DEGs (DNA_REPi)

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


858

In [27]:
# assign GDSC data to new variables

exprs_z = GDSCE
mut = GDSCM
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1659818, 1659818, 1659818, 1659818, 1659819, 1659819, 1659819, 1659819,
       1659819,  687800],
      dtype='int64', length=9435)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
lung_NSCLC           1037
urogenital_system     926
leukemia              829
lymphoma              769
aero_dig_tract        727
nervous_system        607
lung_SCLC             569
skin                  560
breast                524
digestive_system      466
large_intestine       447
bone                  421
neuroblastoma         332
pancreas              294
kidney                227
soft_tissue           191
myeloma               182
lung                  174
thyroid               153
Name: count, dtype: int64

In [32]:
# list count of cell lines as to tissues treated with doxorubicin

filter = (dat["drug"] == "Doxorubicin")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

GDSC\nTissue descriptor 1
lung_NSCLC           86
urogenital_system    76
leukemia             64
aero_dig_tract       61
lymphoma             60
nervous_system       48
skin                 47
lung_SCLC            46
breast               44
digestive_system     38
large_intestine      35
bone                 34
neuroblastoma        26
pancreas             25
kidney               19
soft_tissue          16
lung                 15
myeloma              13
thyroid              13
Name: count, dtype: int64

In [33]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [34]:
# filter GDSC expression and mutation datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    m = mut.loc[samples, :]
    expression_zscores.append(e_z)
    mutations.append(m)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCMv2 = pd.concat(mutations, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCMv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCMv2 = GDSCMv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEdoxorubicin = TCGAEdoxorubicin.loc[:,ls4]
TCGAMdoxorubicin = TCGAMdoxorubicin.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCMv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for Doxorubicin (Expression): {TCGAEdoxorubicin.shape}")
print(f"TCGA # of common samples and genes for Doxorubicin (Mutation): {TCGAMdoxorubicin.shape}\n")

GDSC # of common samples and genes (Expression): (9435, 831)
GDSC # of common samples and genes (Mutation): (9435, 831)
GDSC # of common samples (Response): 9435

TCGA # of common samples and genes for Doxorubicin (Expression): (18, 831)
TCGA # of common samples and genes for Doxorubicin (Mutation): (18, 831)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 0 1]


In [36]:
# change directory to read TCGA response dataset (Doxorubicin)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read TCGA response dataset (Doxorubicin)

TCGARdoxorubicin = pd.read_csv("TCGA_response.Doxorubicin.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARdoxorubicin.dropna(inplace=True)
TCGARdoxorubicin.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARdoxorubicin["response"] = TCGARdoxorubicin.loc[:, "response"].apply(lambda x: d[x])
Ytsdoxorubicin = TCGARdoxorubicin["response"].values    

print(f"There are {Ytsdoxorubicin.shape[0]} samples with response data in the TCGA (Doxorubicin)\n")
Ytsdoxorubicin

There are 18 samples with response data in the TCGA (Doxorubicin)



array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [38]:
# TCGA cohort table with TCGA response dataset (Doxorubicin)

dat_tcga = TCGARdoxorubicin["cohort"]

dat_tcga.value_counts()

cohort
SARC    15
THCA     2
UCS      1
Name: count, dtype: int64

In [39]:
# filter SARC cohort samples (Doxorubicin)

filter = (TCGARdoxorubicin["cohort"] == "SARC")
tcga_sarc_samples = dat_tcga.loc[filter,].index
tcga_sarc_samples

Index(['TCGA-3B-A9I0-01A', 'TCGA-3B-A9I1-01A', 'TCGA-DX-A6B8-01A',
       'TCGA-DX-A6YX-01A', 'TCGA-DX-A7EQ-01A', 'TCGA-DX-A8BU-01A',
       'TCGA-HB-A43Z-01A', 'TCGA-IE-A3OV-01A', 'TCGA-IF-A4AJ-01A',
       'TCGA-IS-A3K7-01A', 'TCGA-K1-A42X-01A', 'TCGA-K1-A6RU-01A',
       'TCGA-LI-A67I-01A', 'TCGA-QQ-A8VD-01A', 'TCGA-X6-A8C6-01A'],
      dtype='object', name='sample')

In [40]:
# filter responses of SARC cohort samples (Doxorubicin)

Ytsdoxorubicin_sarc = TCGARdoxorubicin.loc[tcga_sarc_samples,"response"].values
Ytsdoxorubicin_sarc

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [41]:
# count of sensitive and resistance SARC samples (Doxorubicin)

TCGARdoxorubicin.loc[tcga_sarc_samples,"response"].value_counts()

response
0    13
1     2
Name: count, dtype: int64

In [42]:
# define maximum iteration

max_iter = 50

In [43]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 64
hdm2 = 32
rate1 = 0.5
rate2 = 0.6
rate3 = 0.6

In [44]:
# load pre-trained modules and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEdoxorubicin = scalerGDSC.transform(TCGAEdoxorubicin.values)    

X_testEdoxorubicin = scalerGDSC.transform(TCGAEdoxorubicin.values)    
X_testEdoxorubicin_sarc = scalerGDSC.transform(TCGAEdoxorubicin.loc[tcga_sarc_samples,].values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_testMdoxorubicin = np.nan_to_num(TCGAMdoxorubicin.values.astype(np.float32))
X_testMdoxorubicin_sarc = np.nan_to_num(TCGAMdoxorubicin.loc[tcga_sarc_samples,].values.astype(np.float32))

TX_testEdoxorubicin = torch.FloatTensor(X_testEdoxorubicin)
TX_testMdoxorubicin = torch.FloatTensor(X_testMdoxorubicin.astype(np.float32))
ty_testEdoxorubicinorouracil = torch.FloatTensor(Ytsdoxorubicin.astype(int))

TX_testEdoxorubicin = torch.FloatTensor(X_testEdoxorubicin)
TX_testEdoxorubicin_sarc = torch.FloatTensor(X_testEdoxorubicin_sarc)

TX_testMdoxorubicin = torch.FloatTensor(X_testMdoxorubicin.astype(np.float32))
TX_testMdoxorubicin_sarc = torch.FloatTensor(X_testMdoxorubicin_sarc.astype(np.float32))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")
AutoencoderM = torch.load(models_dir + "Mut_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")

Clas = torch.load(models_dir + "Class_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")

AutoencoderE.eval()
AutoencoderM.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZTX = torch.cat((ZEX, ZMX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_DNA_REPi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n")

ZETdoxorubicin = AutoencoderE(TX_testEdoxorubicin)
ZMTdoxorubicin = AutoencoderM(TX_testMdoxorubicin)
ZTTdoxorubicin = torch.cat((ZETdoxorubicin, ZMTdoxorubicin), 1)
ZTTdoxorubicin = F.normalize(ZTTdoxorubicin, p=2, dim=0)
PredTdoxorubicin = Clas(ZTTdoxorubicin)
AUCTdoxorubicin = roc_auc_score(Ytsdoxorubicin.astype(int), PredTdoxorubicin.detach().numpy())
#print(f"TCGA - Test AUC (Doxorubicin): {round(AUCTdoxorubicin,2)}")
AUCTdoxorubicin_aucpr = average_precision_score(Ytsdoxorubicin.astype(int), PredTdoxorubicin.detach().numpy())
#print(f"TCGA - Test AUCPR (Doxorubicin): {round(AUCTdoxorubicin_aucpr,2)}\n")

ZETdoxorubicin_sarc = AutoencoderE(TX_testEdoxorubicin_sarc)
ZMTdoxorubicin_sarc = AutoencoderM(TX_testMdoxorubicin_sarc)
ZTTdoxorubicin_sarc = torch.cat((ZETdoxorubicin_sarc, ZMTdoxorubicin_sarc), 1)
ZTTdoxorubicin_sarc = F.normalize(ZTTdoxorubicin_sarc, p=2, dim=0)
PredTdoxorubicin_sarc = Clas(ZTTdoxorubicin_sarc)
AUCTdoxorubicin_sarc = roc_auc_score(Ytsdoxorubicin_sarc.astype(int), PredTdoxorubicin_sarc.detach().numpy())
#print(f"TCGA - Test AUC (Doxorubicin, SARC): {round(AUCTdoxorubicin_sarc,2)}")
AUCTdoxorubicin_sarc_aucpr = average_precision_score(Ytsdoxorubicin_sarc.astype(int), PredTdoxorubicin_sarc.detach().numpy())
#print(f"TCGA - Test AUCPR (Doxorubicin, SARC): {round(AUCTdoxorubicin_sarc_aucpr,2)}")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTdoxorubicin_finetuned = dict()
AUCTdoxorubicin_sarc_finetuned = dict()
AUCTdoxorubicin_aucpr_finetuned = dict()
AUCTdoxorubicin_sarc_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   AutoencoderM_finetuned = torch.load(models_dir + f"Finetuned_Models/Mut_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderM_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZMX = AutoencoderM_finetuned(torch.FloatTensor(X_trainM))
   ZTX = torch.cat((ZEX, ZMX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETdoxorubicin = AutoencoderE_finetuned(TX_testEdoxorubicin)
   ZMTdoxorubicin = AutoencoderM_finetuned(TX_testMdoxorubicin)
   ZTTdoxorubicin = torch.cat((ZETdoxorubicin, ZMTdoxorubicin), 1)
   ZTTdoxorubicin = F.normalize(ZTTdoxorubicin, p=2, dim=0)
   PredTdoxorubicin = Clas_finetuned(ZTTdoxorubicin)
   AUCTdoxorubicin_finetuned[iter] = roc_auc_score(Ytsdoxorubicin.astype(int), PredTdoxorubicin.detach().numpy())
   AUCTdoxorubicin_aucpr_finetuned[iter] = average_precision_score(Ytsdoxorubicin.astype(int), PredTdoxorubicin.detach().numpy())

   ZETdoxorubicin_sarc = AutoencoderE_finetuned(TX_testEdoxorubicin_sarc)
   ZMTdoxorubicin_sarc = AutoencoderM_finetuned(TX_testMdoxorubicin_sarc)
   ZTTdoxorubicin_sarc = torch.cat((ZETdoxorubicin_sarc, ZMTdoxorubicin_sarc), 1)
   ZTTdoxorubicin_sarc = F.normalize(ZTTdoxorubicin_sarc, p=2, dim=0)
   PredTdoxorubicin_sarc = Clas_finetuned(ZTTdoxorubicin_sarc)
   AUCTdoxorubicin_sarc_finetuned[iter] = roc_auc_score(Ytsdoxorubicin_sarc.astype(int), PredTdoxorubicin_sarc.detach().numpy())
   AUCTdoxorubicin_sarc_aucpr_finetuned[iter] = average_precision_score(Ytsdoxorubicin_sarc.astype(int), PredTdoxorubicin_sarc.detach().numpy())

max_key = max(AUCTdoxorubicin_aucpr_finetuned, key=AUCTdoxorubicin_aucpr_finetuned.get)
max_key_sarc = max(AUCTdoxorubicin_sarc_aucpr_finetuned, key=AUCTdoxorubicin_sarc_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Doxorubicin): {round(AUCTdoxorubicin,2)}")
print(f"TCGA - Test AUCPR (Doxorubicin): {round(AUCTdoxorubicin_aucpr,2)}")
print(f"TCGA - Test AUC (Doxorubicin, SARC): {round(AUCTdoxorubicin_sarc,2)}")
print(f"TCGA - Test AUCPR (Doxorubicin, SARC): {round(AUCTdoxorubicin_sarc_aucpr,2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Doxorubicin, Finetuned): {round(AUCTdoxorubicin_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Doxorubicin, Finetuned): {round(AUCTdoxorubicin_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_sarc],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_sarc],2)}")
print(f"TCGA - Test AUC (Doxorubicin, SARC, Finetuned): {round(AUCTdoxorubicin_sarc_finetuned[max_key_sarc],2)}")
print(f"TCGA - Test AUCPR (Doxorubicin, SARC, Finetuned): {round(AUCTdoxorubicin_sarc_aucpr_finetuned[max_key_sarc],2)}\n\n")

print("Models for maximum finetuning (Doxorubicin)")
print(f"Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Mut_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Class_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (Doxorubicin, SARC)")
print(f"Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_sarc}.pt")
print(f"Mut_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_sarc}.pt")
print(f"Class_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_sarc}.pt")

GDSC - Training AUC (DNA_REPi): 0.8
GDSC - Training AUCPR (DNA_REPi): 0.31
TCGA - Test AUC (Doxorubicin): 0.62
TCGA - Test AUCPR (Doxorubicin): 0.41
TCGA - Test AUC (Doxorubicin, SARC): 0.46
TCGA - Test AUCPR (Doxorubicin, SARC): 0.32

GDSC - Training AUC (DNA_REPi, Finetuned): 0.81
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.32
TCGA - Test AUC (Doxorubicin, Finetuned): 0.88
TCGA - Test AUCPR (Doxorubicin, Finetuned): 0.67

GDSC - Training AUC (DNA_REPi, Finetuned): 0.81
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.32
TCGA - Test AUC (Doxorubicin, SARC, Finetuned): 0.88
TCGA - Test AUCPR (Doxorubicin, SARC, Finetuned): 0.7


Models for maximum finetuning (Doxorubicin)
Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_46.pt
Mut_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_46.pt
Class_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_46.pt

Models for maximum finetuning (Doxorubicin, SARC)
Exprs_Doxorubicin_GDSC_TCGA_DNA_REPi_Expression_Mu

In [45]:
# create a dataframe with the prediction results 

em = {"Data": ["Expr + Mut", "Expr + Mut (Fine-tuned)"],
      "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
      "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
      "AUC (TCGA)": [round(AUCTdoxorubicin,2), round(AUCTdoxorubicin_finetuned[max_key],2)],
      "AUCPR (TCGA)":  [round(AUCTdoxorubicin_aucpr,2), round(AUCTdoxorubicin_aucpr_finetuned[max_key],2)],
      "AUC (TCGA-SARC)":  [round(AUCTdoxorubicin_sarc,2), round(AUCTdoxorubicin_sarc_finetuned[max_key_sarc],2)],
      "AUCPR (TCGA-SARC)": [round(AUCTdoxorubicin_sarc_aucpr,2), round(AUCTdoxorubicin_sarc_aucpr_finetuned[max_key_sarc],2)],
      "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
      "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
em_dataframe = pd.DataFrame.from_dict(em, orient='index').transpose() 

em_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_Mutation_DNA_REPi_Doxorubicin.tsv",
                    sep = "\t",
                    index = False)

em_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-SARC),AUCPR (TCGA-SARC),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + Mut,0.8,0.31,0.62,0.41,0.46,0.32,9435,831
1,Expr + Mut (Fine-tuned),0.81,0.32,0.88,0.67,0.88,0.7,9435,831


In [46]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=831, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [47]:
# show mutation layer parameters

AutoencoderM

AEM(
  (EnM): Sequential(
    (0): Linear(in_features=831, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [48]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=96, out_features=1, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Sigmoid()
  )
)

In [49]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Predictions


In [50]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_DNA_REPi_Doxorubicin Predictions (Expression and Mutation).txt', 'w')
file.write(f"GDSC Training (EM) AUC (TCGA_DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EM) AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EM) AUC (Doxorubicin): {round(AUCTdoxorubicin,2)}\n")
file.write(f"TCGA Test (EM) AUCPR (Doxorubicin): {round(AUCTdoxorubicin_aucpr,2)}\n\n")

file.close()